# DashScope API Key Connectivity Test

This notebook verifies your `DASHSCOPE_API_KEY` against both the Mainland (CN) and International (INTL) OpenAI-compatible endpoints.

What it does:
- Loads `.env` and reads `DASHSCOPE_API_KEY`.
- Tests HTTP POST to `/chat/completions` on both base URLs.
- Optionally tests via the OpenAI client with `base_url` override.

Expected: 200 response with a completion. If you see 401 invalid_api_key on one URL and 200 on the other, use the URL that returns 200.

In [3]:
!pip install python-dotenv requests openai

Defaulting to user installation because normal site-packages is not writeable
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
   ---------------------------------------- 0.0/786.8 kB ? eta -:--:--
   --------------------------------------- 786.8/786.8 kB 16.9 MB/s eta 0:00:00
Using cached httpx-0.28.1-py3-none-any.whl (73 kB)
Using cached idna-3.10-py3-none-any.whl (70 kB)
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 2.0/2.0 MB 27.3 MB/s eta 0:00:00
Using cached sniffio-1.3.1-py3-none-any.whl (10 kB)



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: C:\Users\haotz\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [4]:
# Load environment and read the API key (masked output)
import os
from pathlib import Path

try:
    from dotenv import load_dotenv
    # Try project root .env
    env_path = Path('.') / '.env'
    if env_path.exists():
        load_dotenv(env_path)
    else:
        load_dotenv()  # fallback to default lookup
except Exception as e:
    print(f'Warning: python-dotenv not available or failed to load .env: {e}')

api_key = os.getenv('DASHSCOPE_API_KEY', '').strip()
masked = api_key[:6] + '...' + api_key[-4:] if len(api_key) > 12 else ('<empty>' if not api_key else api_key[:3] + '...')
print('DASHSCOPE_API_KEY loaded:', masked)
assert api_key, 'DASHSCOPE_API_KEY is missing; set it in .env or environment before testing.'

DASHSCOPE_API_KEY loaded: sk-6ef...b7bf


In [5]:
# Define test helpers and endpoints
import json, requests

CN_BASE = 'https://dashscope.aliyuncs.com/compatible-mode/v1'
INTL_BASE = 'https://dashscope-intl.aliyuncs.com/compatible-mode/v1'

def test_endpoint(base_url: str, key: str, model: str = 'qwen-turbo', timeout: int = 15):
    url = base_url.rstrip('/') + '/chat/completions'
    headers = {
        'Authorization': f'Bearer {key}',
        'Content-Type': 'application/json'
    }
    payload = {
        'model': model,
        'messages': [{ 'role': 'user', 'content': 'ping' }],
        'max_tokens': 10,
        'temperature': 0.1
    }
    try:
        r = requests.post(url, headers=headers, json=payload, timeout=timeout)
        print(f'POST {url} -> {r.status_code}')
        # Show compact error or partial content
        text = r.text
        print(text[:400] + ('...' if len(text) > 400 else ''))
        return r
    except Exception as e:
        print(f'HTTP request failed: {e}')
        return None

print('Testing raw HTTP against CN and INTL endpoints...')
resp_cn = test_endpoint(CN_BASE, api_key)
resp_intl = test_endpoint(INTL_BASE, api_key)

def status_summary(resp):
    if resp is None:
        return 'no response'
    try:
        data = resp.json()
    except Exception:
        return f'status={resp.status_code}, non-JSON body'
    err = data.get('error') if isinstance(data, dict) else None
    if err:
        return f'status={resp.status_code}, code={err.get('code')}, message={err.get('message')}'
    return f'status={resp.status_code}, ok'

print('CN summary:', status_summary(resp_cn))
print('INTL summary:', status_summary(resp_intl))

Testing raw HTTP against CN and INTL endpoints...
POST https://dashscope.aliyuncs.com/compatible-mode/v1/chat/completions -> 401
{"error":{"message":"Incorrect API key provided. ","type":"invalid_request_error","param":null,"code":"invalid_api_key"},"request_id":"01d47d5f-8241-9431-b9f2-c840fdc05c59"}
POST https://dashscope.aliyuncs.com/compatible-mode/v1/chat/completions -> 401
{"error":{"message":"Incorrect API key provided. ","type":"invalid_request_error","param":null,"code":"invalid_api_key"},"request_id":"01d47d5f-8241-9431-b9f2-c840fdc05c59"}
POST https://dashscope-intl.aliyuncs.com/compatible-mode/v1/chat/completions -> 200
{"choices":[{"message":{"role":"assistant","content":"Hello! It seems like you might have intended to"},"finish_reason":"length","index":0,"logprobs":null}],"object":"chat.completion","usage":{"prompt_tokens":13,"completion_tokens":10,"total_tokens":23,"prompt_tokens_details":{"cached_tokens":0}},"created":1755221312,"system_fingerprint":null,"model":"qwen-t

In [6]:
# Optional: Test via OpenAI client with base_url override
try:
    from openai import OpenAI
    messages = [{ 'role': 'user', 'content': 'Say hello in one sentence.' }]
    for base in (CN_BASE, INTL_BASE):
        try:
            client = OpenAI(api_key=api_key, base_url=base)
            print(f'OpenAI client test -> {base}')
            r = client.chat.completions.create(model='qwen-turbo', messages=messages, max_tokens=20, temperature=0.1)
            # Print compact result
            content = r.choices[0].message.content if getattr(r, 'choices', None) else str(r)
            print('OK:', (content[:200] + '...') if content and len(content) > 200 else content)
        except Exception as e:
            print('OpenAI client error:', e)
except Exception as e:
    print('OpenAI package not available or failed to import:', e)

OpenAI client test -> https://dashscope.aliyuncs.com/compatible-mode/v1
OpenAI client error: Error code: 401 - {'error': {'message': 'Incorrect API key provided. ', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}, 'request_id': '77d0b00e-ed72-9745-8b27-d3fd2c407356'}
OpenAI client error: Error code: 401 - {'error': {'message': 'Incorrect API key provided. ', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}, 'request_id': '77d0b00e-ed72-9745-8b27-d3fd2c407356'}
OpenAI client test -> https://dashscope-intl.aliyuncs.com/compatible-mode/v1
OpenAI client test -> https://dashscope-intl.aliyuncs.com/compatible-mode/v1
OK: Hello! How can I assist you today?
OK: Hello! How can I assist you today?
